# Red Transformer BERT

Para las tareas como esta de detectar noticias falsas, los modelos tradicionales se ha quedado un poco atras en comparación con la red Transformers como BERT. 
Las redes transformers son la revolución del NLP de los ultimos años, son modelos de deep learning diseñado para procesar secuencias de datos, especialmente texto, de manera eficiente y precisa.
A diferencia de las RNNs, que procesan las palabras de manera secuencial (una por una), las Transformers procesan todas las palabras al mismo tiempo (paralelo) gracias a un mecanismo llamado Self-Attention, es decir, entendiendo su contexto.
En un modelo tradicional (RNN):
Se procesa palabra por palabra de izquierda a derecha.
Problema: La relación entre "gato" y "ratón" puede perderse debido a la distancia en la oración.
 En una Transformer: Cada palabra ve todas las demás simultáneamente. Se aplican pesos de atención para determinar qué palabras son más importante en el contexto.

BERT Esta basado en estas rede s(Bidirectional Encoder Representations from Transformers) es un modelo de procesamiento del lenguaje natural (NLP) desarrollado por Google en 2018, con un corpues gigantesto  de wikipedia y google. Es revolucionario porque entiende el contexto de una palabra en ambos sentidos (izquierda y derecha) en una oración, lo que lo hace extremadamente poderoso para tareas como la detección de noticias falsas. Por ejemplo, BERT entedera que la palabra "banco" cambia según el contexto. Es considerado uno de las revoluciones de los ultimos años en NLP debido a que rompió la barrera para la comunicación entre las personas y los ordenadores, como por ejemplo a la hora de analizar sentimientos.

¿Cómo funciona BERT?
FASE 1
1.  Preentrenamiento en dos tareas:
Masked Language Model (MLM):
- BERT oculta ("mask") palabras al azar y trata de predecirlas.
2. Next Sentence Prediction (NSP): Aprende relaciones entre oraciones consecutivas.
Esto permite que BERT entienda la estructura del lenguaje a nivel profundo.

FASE 2
1. afinación: se agrega 2 capas, una de red neuronal y otra de softmax y se presentan las frases reentrenandolas.


¿ Por qué es bueno en predicción de noticias falsas?

Detectar noticias falsas es una tarea de clasificación de texto, en la que BERT puede ser mucho mas recomendado que los modelos tradicionales.

1. Comprende el contexto mejor que otros modelos

- Noticias falsas suelen usar lenguaje engañoso: BERT puede detectar contradicciones y diferencias de tono.

2. Puede entrenarse en datasets de noticias verdaderas y falsas: Ajustando (fine-tuning) BERT en datos etiquetados, aprende a distinguir patrones típicos de fake news.

3. Identifica relaciones semánticas entre frases: Muchas noticias falsas manipulan información de otras fuentes.
BERT puede detectar inconsistencias entre afirmaciones dentro del mismo artículo.

4. Funciona bien con grandes volúmenes de texto:  diferencia de modelos tradicionales (TF-IDF, Naive Bayes), BERT no necesita preprocesamiento intenso.




### instalación de Bert en google colab

In [2]:
!pip install torch transformers scikit-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 105.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 85.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 43.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

## Carga de datos

In [1]:
import pandas as pd
df = pd.read_csv("/content/noticias_fake_true.csv")


Los datos que le paso es sin transformaciones previas debido a que Bert analiza el texto completo y los preprocesamientos pueden afectar a la interpretacion del modelo al quitarle información

## Librerías

In [3]:
import torch # para construir y entrenar modelos de deep learning de redes neuronales
from torch.utils.data import DataLoader, Dataset # definir capas neuronales
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd

#  Verificar si hay GPU disponible y mover modelo a GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Dispositivo usado:", device)


#  Preprocesamiento: Usar solo el texto y la etiqueta
X = df['text'].fillna(" ")  # Rellenamos valores nulos con texto vacío
y = df['label']

# Dividir en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) # Para mantener la reporducibilidad

# Cargar un modelo más liviano: DistilBERT
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2).to(device)

# Tokenizar los textos
def encode_texts(texts, tokenizer, max_length=256):
    return tokenizer(list(texts), padding=True, truncation=True, max_length=max_length, return_tensors="pt") 
    # padding para que tenga la misma longitud.
    # truncation para acortar los textos demasiados largos
    #return_tensors="pt"= convierte  los tokens en Pytorch( lo que espera el modelo transformer)

#Utiliza la funcion encode para transformas X_train y X_test
train_encodings = encode_texts(X_train, tokenizer) 
test_encodings = encode_texts(X_test, tokenizer)

# Crear Dataset personalizado
class NewsDataset(Dataset): #convierte los tokens y etiquetas (labels) en un objeto compatible con PyTorch, que luego será usado por DataLoader para alimentar el modelo en lotes (batches).
    def __init__(self, encodings, labels):
        self.encodings = encodings # Contiene los textos tokenizados (input_ids, attention_mask)
        self.labels = labels        # Contiene las etiquetas (0 o 1 en clasificación binaria)
        self.labels = labels

    def __len__(self): # Para que Dataloader sepa cuantos datos hay en dataset 
        return len(self.labels)# Retorna la cantidad de muestras en el dataset

    def __getitem__(self, idx): # obtiene los tokens para la muestra 
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels.iloc[idx]) # convierte label en Pytorch 
        return item
# Carga del dataset en Pytorch y lo convierte en un formato compatible con DataLoader
train_dataset = NewsDataset(train_encodings, y_train)
test_dataset = NewsDataset(test_encodings, y_test)

# Usar batch_size más pequeño para evitar problemas de memoria
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True) 
# divide en batch para que no colapse 
#  Mezcla los datos en cada epoch para mejorar el entrenamiento.
test_loader = DataLoader(test_dataset, batch_size=4) #Se entrenan 4 muestras a la vez para evitar problemas de memoria.

#  Definir optimizador y función de pérdida
optimizer = AdamW(model.parameters(), lr=2e-5) 
#un optimizador diseñado para modelos Transformers como BERT.
#model.parameters() → Obtiene los pesos del modelo que serán ajustados.
#lr=2e-5 (learning rate) → Controla qué tan rápido el modelo ajusta sus pesos.

loss_fn = torch.nn.CrossEntropyLoss() #Define la función de pérdida (loss function), que mide qué tan mal está prediciendo el modelo.

# Función de entrenamiento
def train_model(model, train_loader, optimizer, loss_fn, device, epochs=2):
    model.train()
    for epoch in range(epochs): # El entrenamiento se repite epochs veces para que el modelo vea los datos varias veces y mejore.
        print(f"Época {epoch+1}/{epochs}")
        total_loss = 0 # total_loss = 0 almacena la suma de pérdidas de cada mini-lote para calcular el promedio al final de la época.
        for batch in train_loader: # Itera sobre los datos de entrenamiento en mini-lotes.
            optimizer.zero_grad() #Limpia los gradientes acumulados de iteraciones previas.
            inputs = {key: val.to(device) for key, val in batch.items() if key != 'labels'} # Se separan labels (las etiquetas reales) del resto de los datos (input_ids y attention_mask).
            labels = batch['labels'].to(device)
            outputs = model(**inputs)
            loss = loss_fn(outputs.logits, labels) # Compara la predicción del modelo (outputs.logits) con la etiqueta real (labels).
            loss.backward() #Calcula los gradientes de cada peso en el modelo usando backpropagation.
            optimizer.step() #Ajusta los pesos del modelo usando el optimizador (AdamW) para reducir la pérdida.
            total_loss += loss.item()# Guarda la pérdida del batch para calcular la pérdida promedio al final de la época.
        print(f"Pérdida promedio: {total_loss / len(train_loader)}")

#  Entrenar modelo (2 épocas para prueba rápida)
train_model(model, train_loader, optimizer, loss_fn, device, epochs=2)

#  Evaluar el modelo
def evaluate_model(model, test_loader, device):
    model.eval()
    #Desactiva capas como Dropout, que afectan la predicción.
    #Evita modificar los pesos del modelo durante la evaluación.
    predictions, true_labels = [], []
    #predictions guardará las etiquetas predichas por el modelo.
    #true_labels guardará las etiquetas reales del dataset de prueba.
    with torch.no_grad():# Desactiva el cálculo de gradientes para reducir el uso de memoria y acelerar la evaluación.
        for batch in test_loader: # Se procesa el dataset de prueba en mini-lotes (batch_size=4), igual que en el entrenamiento.
            inputs = {key: val.to(device) for key, val in batch.items() if key != 'labels'}
            labels = batch['labels'].to(device) #Convierte los datos a cuda si hay GPU disponible para hacer las predicciones más rápido.
            outputs = model(**inputs) #Ejecuta el modelo con los datos de prueba y obtiene los logits de salida.
            preds = torch.argmax(outpsuts.logits, dim=1).cpu().numpy() #Convierte los logits en etiquetas de clase (0 o 1) usando argmax
            predictions.extend(preds) #Se almacenan las predicciones (preds) y las etiquetas reales (true_labels) para calcular métricas.
            true_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(true_labels, predictions)
    print(f"Precisión del modelo: {accuracy:.4f}")
    print("Reporte de clasificación:")
    print(classification_report(true_labels, predictions))

#  Ejecutar evaluación
evaluate_model(model, test_loader, device)


Dispositivo usado: cuda


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Época 1/2
Pérdida promedio: 0.007961040178913678
Época 2/2
Pérdida promedio: 0.0011333290111552625
Precisión del modelo: 0.9994
Reporte de clasificación:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      4650
           1       1.00      1.00      1.00      4330

    accuracy                           1.00      8980
   macro avg       1.00      1.00      1.00      8980
weighted avg       1.00      1.00      1.00      8980



He utilizado DistilBert  ya que tenía algunos problemas computacionales. Distil Bert es una versión optimizada y especializada para clasificación de texto, tiene menos parametros pero se pueden obtener buenos resultados.

## Interpretación resultados

### Reporte de clasificación

Clase 0 (Noticias Falsas) y Clase 1 (Noticias Verdaderas) tiene un 100% en todas las métricas. Todas las predicciones coinciden exactamente con las etiquetas verdaderas.


Precisión (accuracy) = 99.94% → De cada 10,000 ejemplos, solo 6 se clasifican incorrectamente.


La  pérdida disminuye significativamente de 0.0079 a 0.0011, lo que indica que el modelo está convergiendo bien.


## Prueba para evaluar el modelo

In [6]:
import torch
from sklearn.metrics import accuracy_score

def evaluate(model, dataloader, device):
    model.eval()
    predictions, true_labels = [], []

    with torch.no_grad():
        for batch in dataloader:
            inputs = {key: val.to(device) for key, val in batch.items() if key != 'labels'}
            labels = batch['labels'].to(device)
            outputs = model(**inputs)
            preds = torch.argmax(outputs.logits, dim=1).cpu().numpy()
            predictions.extend(preds)
            true_labels.extend(labels.cpu().numpy())

    return accuracy_score(true_labels, predictions)

# Calcular precisión en entrenamiento
train_accuracy = evaluate(model, train_loader, torch.device("cuda" if torch.cuda.is_available() else "cpu"))

# Calcular precisión en prueba
test_accuracy = evaluate(model, test_loader, torch.device("cuda" if torch.cuda.is_available() else "cpu"))

# Imprimir resultados
print(f"📊 Precisión en entrenamiento: {train_accuracy:.4f}")
print(f"📊 Precisión en prueba: {test_accuracy:.4f}")


📊 Precisión en entrenamiento: 0.9997
📊 Precisión en prueba: 0.9994


1.  Precisión en entrenamiento = 99.97%

Esto indica que el modelo ha aprendido a clasificar correctamente casi todos los ejemplos en el conjunto de entrenamiento.
Muy baja pérdida (loss) → El modelo ha minimizado el error casi por completo.
2. Precisión en prueba = 99.94%

La precisión en el conjunto de prueba es casi igual a la de entrenamiento, lo que indica que no hay un overfitting severo.
Si hubiera overfitting, la precisión en prueba sería mucho más baja que en entrenamiento.